In [ ]:
import pandas as pd

df = pd.read_csv('C://Users//and_c//Desktop//sparta_ga4//train_df3.csv', low_memory = False)

In [ ]:
# 해당 값 전체 보기
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.options.display.max_columns = None

In [ ]:
df.info()

In [ ]:
# 데이터프레임에 한 열의 값을 바꾸기 위해서 def함수를 만들고 apply 적용
import pandas as pd
import datetime #날짜와 시간을 다루는 라이브러리
import pytz #다양한 시간대를 처리하기 위해 사용하는 라이브러리

# Posix 시간을 utc 시간으로 변환하는 함수
def posix_to_utc(visitStartTime):
    utc_time = datetime.datetime.utcfromtimestamp(visitStartTime)
    return utc_time

# Posix 시간을 UTC 지정한 로컬 시간대로 변환하는 함수
def posix_to_utc_local(visitStartTime):
    utc_time = datetime.datetime.utcfromtimestamp(visitStartTime) #posix -> utc시간대로
    local_timezone = pytz.timezone('America/New_York')  # 지역 시간대
    return utc_time.astimezone(local_timezone) # 그 지역 시간대로 변환

In [ ]:
df['visitStartTime'] = df['visitStartTime'].apply(posix_to_utc)

DAU / WAU / MAU를 Line Char로 시각화

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df['year'] = df['visitStartTime'].dt.year

In [ ]:
# DAU구하기
dau_df = df.groupby(df['visitStartTime'].dt.date)[['fullVisitorId']].nunique()

plt.figure(figsize = (8,5))
plt.plot(dau_df.index, dau_df['fullVisitorId'])
plt.xlabel('date')
plt.ylabel('dau')
plt.xticks(fontsize=8)
plt.show()

In [ ]:
# WAU구하기
wau_df = df.groupby(df['visitStartTime'].dt.week)[['fullVisitorId']].nunique()

plt.figure(figsize = (8,5))
plt.plot(wau_df.index, wau_df['fullVisitorId'])
plt.xlabel('date')
plt.ylabel('wau')
plt.xticks(fontsize=8)
plt.show()

In [ ]:
# 월을 기준으로 계산
month_df = df.groupby(df['visitStartTime'].dt.month)[['fullVisitorId']].nunique()

plt.figure(figsize = (8,5))
plt.plot(month_df.index, month_df['fullVisitorId'])
plt.xlabel('month')
plt.ylabel('month_num')
plt.xticks(fontsize=8)
plt.show()

- 10~12월에 접속량 증가

유저별 평균접속시간 분포 ECDF로 시각화

In [ ]:
# date칼럼 추가
df['date'] = df['visitStartTime'].dt.strftime('%Y-%m-%d')

In [ ]:
def min_max(x):
    return x.max()-x.min()

In [ ]:
df.groupby(['fullVisitorId','date'])['visitStartTime'].agg(min_max)

- ex. 1시간 혹은 2~3시간 뒤에 다시 접속한 경우가 있으면..?

방문 주차에 따른 코호트를 생성하고, 코호트 별로 weekly 리텐션을 구하고 이를 heatmap으로 시각화

In [ ]:
# week 칼럼 만들기
df['week'] = df['visitStartTime'].dt.week

In [ ]:
week_df = df.groupby('week')['fullVisitorId'].count().reset_index()

In [ ]:
a = list(week_df['fullVisitorId'])
b = list(week_df['week'])

In [ ]:
#리텐션=(이전 주에 활동한 사용자 수/해당 주에 활동한 사용자 수)×100
# newvisit을 
week_co = []
for i in range(1,52):
    week_co.append(a[i-1]/a[i]*100)

In [ ]:
week_co_df = pd.DataFrame({'week':b[1:], 'Week_retention':week_co})
week_co_df

In [ ]:
table = week_co_df.pivot_table(index='week', values='Week_retention')

plt.rcParams['figure.figsize'] = [10,8]
plt.pcolor(table)
# 또는 sns.heatmap()
# 피벗테이블의 df를 pandas를 이용해 표현도 가능 df.style.background_gradient
plt.title('Weekly Retention', fontsize=20)
plt.xlabel('week', fontsize=14)
plt.ylabel('visit', fontsize=14)
plt.colorbar()

Carrying Capapcity

In [ ]:
# Wau / (1-retention)


국가별 DAU 평균을 Folium으로 시각화하기

In [ ]:
df.groupby('geoNetwork_continent')

캠페인 효과 분석: 캠페인으로 들어온 사람은 다른 사람과 다른 특징을 가지고 있는지 분석해보기